In [1]:
## Import libs
import findspark
findspark.init("/home/antonis/spark-2.3.0-bin-hadoop2.7")

import os.path
import pandas
import math
import time

from metrics.Correlation import Correlation
from IO.Output import Output
from IO.Input import Input
from metrics.RSquare import RSquare
from DataSet.Dataset import DataSet

import pyspark
from pyspark.sql import SparkSession


import sys
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql.functions import countDistinct,avg,stddev
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                               OneHotEncoder , StringIndexer)
import DataSet.SnpsSelection as s

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.linalg import DenseVector
from pyspark.mllib.evaluation import MulticlassMetrics

from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LinearSVC

from pyspark.ml.classification import NaiveBayes

from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils
from pyspark.mllib.tree import GradientBoostedTrees, GradientBoostedTreesModel


import math
import time


def writeCoef(path,snpsIds,sc,idToName,name = None):
        
    if not name:
        print("give a name to file")
        return
        
    p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ).txt "  
    
    i=1
    while os.path.exists(p):
            
        p = path + name  + " ( " + time.strftime("%d-%m-%Y") + " ) " + '_' + str(i)+".txt"
        i += 1
        
    snps = []
    for i in range(len(snpsIds)):
        s = snpsIds[i]
        #snps.append(idToName[s])
        snps.append(s)
            
    print("snpsIds = ",len(snpsIds))
    print("idToName = ",len(idToName))
        
    write = open(p,'w')
    for i in range(len(snps)):
            
        write.write(str(snps[i])+'\t'+str(sc[i])+'\n')
            
    write.close()


def calcCoeff(path,coefs, columns):
    
    ids = {}
    ids['coef']={}
   
    print("before = ",len(set(coefs)))
    #for i in range(len(coefs)):
     #   coefs[i] = abs(coefs[i])

    print("after = ",len(set(coefs)))

    idToName = {}
    nameToId = {}

    for i in range(len(coefs)):
        nameToId[coefs[i]] = []

    for i in range(len(coefs)):
        nameToId[coefs[i]].append(i)
        idToName[i] = coefs[i]


    ids['coef']['nameToId'] = nameToId
    ids['coef']['idToName'] = idToName


    sc = sorted(coefs,reverse=True)

    top_30 = []

    for i in range(30):

        snp = ids['coef']['nameToId'][sc[i]][0]
        ids['coef']['nameToId'][sc[i]].remove(snp)
        top_30.append(columns[snp])
     #   top_30.append(snp)

    #snpReduc['low'] = top_30
    writeCoef(path,top_30,sc,columns, 'mycutoffabs')
    
    
def showMetrics(c,value=1):
    
    predictionAndLabels = c.map(lambda lp: (lp.prediction, float(lp.label)))
    metrics = MulticlassMetrics(predictionAndLabels)

    print("confusion matrix = ", metrics.confusionMatrix().toArray())

    print("accuracy = ", metrics.accuracy)
    print("recal = ", metrics.recall(value))
    print("precision = ", metrics.precision(value))
    print("f1 = ", metrics.fMeasure(1.0))
    
    
def showMyMetric(results,v1 = 1,v2 = 0):
    
    if v1 == v2:
        print("wrong values!!!v1 is the same with v2!!!!")
       
    
    r = results.rdd
    r3 = r.collect()

    same0 = 0
    same1 = 0
    sum0 = 0
    sum1 = 0
    sumall = 0
    for i in r3:

        if i[v1] == 0:
            sum0 += 1
            if i[v2] == 0:
                same0 += 1
        elif i[v1] == 1:
            sum1 += 1
            if i[v2] == 1:
                same1 += 1
        sumall += 1

    print('sum0 = ', sum0)
    print('sum1 = ', sum1)
    print('same0 = ', same0)
    print('same1 = ', same1)
    print('all = ', sumall)
    print('all2 = ', sum0+sum1)

In [2]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
    .set("spark.driver.maxResultSize", "20g")
    .set('spark_executor_cores',"3")
    .set('spark.graphx.pregel.checkpointInterval','-1')
    .set('spark.network.timeout','100000000')
    .set('spark.executor.heartbeatInterval','10000000'))

#spark = SparkSession.builder.appName('melanoma').getOrCreate()
sc = SparkContext(conf=conf)
spark = SQLContext(sc)

In [3]:
path = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/'

#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.001/snpCodeTest.csv'
#pathSnp = '/media/antonis/red/newSet/maf/maf = 0.05/assoc/pvalue = 0.01/snp2.txt'
pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'
#pathSnp = '/media/antonis/red/newdata/maf = 0.05/pvalue = 0.001/snpCodeTest1.csv'

#pathSnp = '/media/antonis/Antonis_Moulopoulos/newSet/pvalue = 0.001/snp1.txt'
#pathSnp = '/media/antonis/Antonis_Moulopoulos/newdata/maf = 0.05/pvalue = 0.001/snp2.txt'


data = spark.read.option("maxColumns", 80000).csv(pathSnp,inferSchema=True,header=True)
data=data.withColumnRenamed('TARGET','label')

In [4]:
print("data columns = ",len(data.columns))

features = []
for i in data.columns:
    if 'rs' in i:
        features.append(i)
        break
print(len(features))

#print((features))

data columns =  12144
1


In [5]:
data.columns[len(data.columns)-1]

'label'

In [6]:
assempler = VectorAssembler(inputCols=features,outputCol='features')
output = assempler.transform(data)
final_data = output.select('features','label')

# CORRELATION

In [ ]:
from pyspark.ml.stat import Correlation
final_data1 = final.select('features')
corr = Correlation.corr(final_data1, "features")
corr = corr.head()[0].toArray()

In [ ]:
snpsRed = []
snpsRed = s.lowCorrelation(corr, threshold=0.7, up=100, down=99)


In [ ]:
snpsRed1 = []
for i in snpsRed:
    snpsRed1.append(features[i])

assempler = VectorAssembler(inputCols=snpsRed1,outputCol='snpsRed')
output = assempler.transform(data)
final_data = output.select('snpsRed','label')
#final_data.show(10)

In [ ]:
final_data = output.select('snpsRed','label')
final_data.show(10)

In [ ]:
train0,test_data = final_data.randomSplit([0.9,0.1],seed=11)

In [7]:
train1 = final_data.filter("label == 1")
train0 = final_data.filter("label == 0")

label1 = train1.count()
label0 = train0.count()

l1 = (label1/2) + ((label1/2) * 0.1)

x = (l1 * 100) / label0


x = x / 100


tr1,tr11 = train1.randomSplit([0.5,0.5],seed=11)
tr0,tr00 = train0.randomSplit([x,1-x],seed=11)

train0 = tr0.union(tr1)
test_data = tr00.union(tr11)

t0 = train0.count()
t1 = test_data.count()

print("label0 = ",label0)
print('label1 = ', label1)
print("l1 = ",l1)
print('train = ', t0)
print('test = ', t1)
print('all = ', t0 + t1)
print('final_data = ',final_data.count())

#train0,test_data = final_data.randomSplit([0.9,0.1])

label0 =  240019
label1 =  2157
l1 =  1186.35
train =  2285
test =  239891
all =  242176
final_data =  242176


In [8]:
log_reg = LogisticRegression(featuresCol='features',labelCol='label',maxIter=10)
lr_model = log_reg.fit(train0)

In [9]:

results = lr_model.transform(test_data)
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluate = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='label')
AUC = evaluate.evaluate(results)
print(AUC)
'''0.7909133853963526'''

'0.7909133853963526'

0.5